# INFORMATION_SCHEMA

- requirements : JOBS_BY_FOLDER & JOBS_BY_ORGANIZATION & JOBS_BY_PROJET (bigquery resource Admin)

## DAILY SLOT USAGE

In [ ]:
with table_a as (
--https://github.com/rho715/bigquery-utils/blob/master/dashboards/system_tables/sql/daily_commitments.sql
WITH
  commitments AS (
    SELECT
      change_timestamp,
      commitment_plan,
      action,
      EXTRACT(DATE FROM change_timestamp) AS start_date,
      -- Compute the stop date of a commitment by looking at the following
      -- change_timestamp and subtracting one day. This works because monthly
      -- and yearly commitments cannot be deleted on the same day
      -- they were created.
      IFNULL(
        LEAD(DATE_SUB(EXTRACT(DATE FROM change_timestamp), INTERVAL 1 DAY))
          OVER (PARTITION BY state ORDER BY change_timestamp),
        CURRENT_DATE()) AS stop_date,
      -- In order to calculate the cumulative slots up to this point, add
      -- the slot count of new commitments (indicated by a CREATE or UPDATE action)
      -- and subtract the slot count of deleted commitments
      SUM(CASE WHEN cccp.action IN ('CREATE', 'UPDATE') THEN cccp.slot_count ELSE cccp.slot_count * -1 END)
        -- Cumulative slots are tracked by their state and carried over from
        -- previous rows
        OVER (
          PARTITION BY state
          ORDER BY change_timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        ) AS slot_cummulative,
      -- In the event that multiple changes occurred in one day, we keep track
      -- of the most recent cumulative value by using row numbers
      ROW_NUMBER()
        OVER (
          PARTITION BY EXTRACT(DATE FROM change_timestamp)
          ORDER BY change_timestamp DESC
        ) AS rn
    FROM
       `region-asia-northeast3`.INFORMATION_SCHEMA.CAPACITY_COMMITMENT_CHANGES_BY_PROJECT AS cccp
    -- In this case, we only want to look at active commitments that are
    -- monthly or annual, not flex
    WHERE
      state = 'ACTIVE'
      AND commitment_plan != 'FLEX'
    ORDER BY change_timestamp
  ),
  results AS (SELECT * FROM commitments WHERE rn = 1),
  days AS (
    -- This subquery is used to fill in the missing days between a commitment
    -- starting and ending so that it can be graphed properly.
    SELECT day
    FROM (
       SELECT
         start_date,
         stop_date
       FROM results
     ), UNNEST(GENERATE_DATE_ARRAY(start_date, stop_date)) day
  )
SELECT TIMESTAMP(day) as date, LAST_VALUE(slot_cummulative IGNORE NULLS) OVER(ORDER BY day) slots,
FROM days
-- Join these results with the cumulative slot count values for each day
LEFT JOIN results
  ON day = DATE(change_timestamp)
)
, 
table_b as (
--https://github.com/rho715/bigquery-utils/blob/master/dashboards/system_tables/sql/daily_utilization.sql
SELECT
  TIMESTAMP_TRUNC(jbo.creation_time, DAY) AS usage_date,
  jbo.reservation_id,
  jbo.project_id,
  jbo.job_type,
  jbo.user_email,
  -- Aggregate total_slots_ms used for all jobs on this day and divide
  -- by the number of milliseconds in a day. Most accurate for days with
  -- consistent slot usage
  SUM(jbo.total_slot_ms) / (1000 * 60 * 60 * 24) AS average_daily_slot_usage
FROM
   `region-asia-northeast3`.INFORMATION_SCHEMA.JOBS_BY_ORGANIZATION jbo
GROUP BY
  usage_date,
  jbo.project_id,
  jbo.job_type,
  jbo.user_email,
  jbo.reservation_id
ORDER BY
  usage_date ASC
)
select table_a.date   
      , table_a.slots 
      , table_b.average_daily_slot_usage
from table_a
left join table_b on table_a.date = table_b.usage_date

## HOURLY SLOT USAGE

In [1]:
with jbf as (--https://github.com/rho715/bigquery-utils/blob/master/dashboards/system_tables/sql/hourly_utilization.sql
SELECT
  -- usage_time is used for grouping jobs by the hour
  -- usage_date is used to separately store the date this job occurred
  TIMESTAMP_TRUNC(TIMESTAMP_ADD(TIMESTAMP(jbf.creation_time), INTERVAL 9 HOUR) , HOUR) AS usage_time,
  EXTRACT(DATE from TIMESTAMP_ADD(TIMESTAMP(jbf.creation_time), INTERVAL 9 HOUR) ) AS usage_date,
  -- jbf.reservation_id,
  jbf.project_id,
  jbf.job_id,
  TIMESTAMP_ADD(TIMESTAMP(jbf.start_time), INTERVAL 9 HOUR) start_time,
  TIMESTAMP_ADD(TIMESTAMP(jbf.end_time), INTERVAL 9 HOUR) end_time,
  jbf.state,
  jbf.total_bytes_processed, 
  jbf.job_type,
  jbf.user_email,
  jbf.error_result.reason error_result_reason,
  jbf.error_result.message error_result_message,
  -- Aggregate total_slots_ms used for all jobs at this hour and divide
  -- by the number of milliseconds in an hour. Most accurate for hours with
  -- consistent slot usage
  SUM(jbf.total_slot_ms) / (1000 * 60 * 60) AS average_hourly_slot_usage
FROM
   `region-asia-northeast3`.INFORMATION_SCHEMA.JOBS_BY_FOLDER jbf
GROUP BY
  usage_time,
  usage_date,
  jbf.project_id,
  jbf.job_id,
  start_time,
  end_time,
  jbf.state,
  jbf.total_bytes_processed, 
  jbf.job_type,
  jbf.user_email,
  jbf.error_result.reason ,
  jbf.error_result.message 
ORDER BY
  usage_time ASC
)
select jbf.*
    , jbp.query
    , case when error_result_reason is null then 'COMPLETE' else 'ERROR' end as job_status 
    , timestamp_diff(jbf.end_time, jbf.start_time, second) as job_duration_seconds
from jbf
LEFT JOIN `wavvedp-prd.tmp_rho715.jobs_by_project` jbp on jbf.job_id = jbp.job_id



SyntaxError: invalid syntax (1582267095.py, line 1)